## <font style="font-family:roboto;color:#455e6c"> Introduction to atomistic simulations with pyiron </font>  

<div class="admonition note" name="html-admonition" style="background:#e3f2fd; padding: 10px">
<font style="font-family:roboto;color:#455e6c"> Sarath Menon, Jörg Neugebauer </font> </br>
<font style="font-family:roboto;color:#455e6c"> Max Planck Institute for Sustainable Materials, Dusseldorf, Germany </font> </br>
<font style="font-family:roboto;color:#455e6c"> <b> Machine Learning Modalities for Materials Science Workshop, Jožef Stefan Institute, Ljubljana </b> </font> </br>
<font style="font-family:roboto;color:#455e6c"> 14 May 2024 </font>
</div>

### <font style="font-family:roboto;color:#455e6c"> Jupyter Crash Course </font>  

1. Select cells by clicking on them.
2. Navigate through with `up` and `down` keys (or `k` and `j` for you vimmers).
3. Press Enter to edit a cell.
4. Press Shift-Enter to execute it.
5. Create new cells above or below the current one with `a` or `b`.
6. Copy, cut and paste them with `c`, `x` and `v`.
7. Press `m` to turn a cell into a markdown cell.
8. See the `Help` in the toolbar for more.

Before the excercise, you should:

* Basic understanding of python especially with numerical libraries like numpy and plotting tools like matplotlib
* Understand how Jupyter Notebooks work (see above)

The aim of this exercise is to make you familiar with:

* A general overview of what pyiron can do
* How to set up atomic structures and run atomistic simulation codes through pyiron

### <font style="font-family:roboto;color:#455e6c"> Importing necessary libraries </font>  
As a first step we import the libraries [numpy](http://www.numpy.org/) for data analysis and [matplotlib](https://matplotlib.org/) for visualization.

In [ ]:
%config IPCompleter.evaluation='unsafe'

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
import numpy as np
import matplotlib.pylab as plt

Fundamentally, we only need to import one module from `pyiron`: the `Project` class

In [ ]:
from pyiron import Project

The Project object introduced below is central in pyiron. It allows to name the project as well as to derive all other objects such as structures, jobs etc. without having to import them. Thus, by code completion *Tab* the respective commands can be found easily.

We now create a pyiron Project named 'tutorial'.

### <font style="font-family:roboto;color:#455e6c"> Working with atomistic structures </font>  

#### <font style="font-family:roboto;color:#455e6c"> Creation of a project instance </font>  

In [ ]:
pr = Project("tutorial")

The project name also applies for the directory that is created for the project. All data generated by this `Project` object resides in this directory.

In [ ]:
pr.path

In [ ]:
pr

The `groups` and `nodes` will be populated later, as we add jobs and sub project to it.

#### <font style="font-family:roboto;color:#455e6c"> Creating atomic structures </font>  

Every atomistic simulation needs an atomic structure. For more details on generating and manipulating structures, please have a look at our [structures example](https://pyiron.readthedocs.io/en/latest/source/notebooks/structures.html). In this section however, we show how to generate and manipulate bulk crystals, surfaces, etc. pyiron's structure class is derived from the popular [`ase` package](https://wiki.fysik.dtu.dk/ase/ase/build/build.html) and any `ase` function to manipulate structures can also be applied here.

Creating a bulk fcc cubic unitcell

In [ ]:
Al_unitcell_cubic = pr.create.structure.bulk('Al', cubic=True)
Al_unitcell_cubic

Creating a super cell.

In [ ]:
Al_supercell_3_3_3 = Al_unitcell_cubic.repeat([3, 3, 3])
Al_supercell_3_3_3.plot3d(particle_size=2)

Creating a vacancy is easy, just delete an atom.

In [ ]:
Al_vacancy = Al_supercell_3_3_3.copy()
del Al_vacancy[0] # Deleting the first atom
print(Al_supercell_3_3_3.get_chemical_formula(), Al_vacancy.get_chemical_formula())
Al_vacancy.plot3d(particle_size=2)

### <font style="font-family:roboto;color:#455e6c"> Running an atomistic calculation using interatomic potentials (with LAMMPS) </font>  


Once we have an atomic structure, we can set up a simulation "job" of any atomistic simulation that is intergrated within pyiron. In this section, we are going to use the popular [LAMMPS code](https://lammps.sandia.gov/).

In [ ]:
# Create a job
job_lammps = pr.create.job.Lammps("lammps_job")

Every atomistic simulation code needs an input atomic structure. We use the Al supercell structure we created earlier

In [ ]:
# Assign an atomic structure to the job
job_lammps.structure = pr.create.structure.bulk('Al', cubic=True).repeat(3)

Once the structure is assigned, an appropriate potential should also be chosen. This list of available for the structure containing Al can be found below.  This list originates from the NIST Interatomic Potential Database.

In [ ]:
# See available potentials
job_lammps.list_potentials()[20:30] # to include the selected potential in the list

In [ ]:
# Choose one of these potentials
job_lammps.potential = "2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1"

At this stage, the computational parameters for the simulation needs to be specified. pyiron parses generic computational parameters into code specific parameters allowing for an easy transition between simulation codes

In [ ]:
# specify calculation details: in this case: MD at 800 K in the NPT ensemble (pressure=0) for 10000 steps
job_lammps.calc_md(temperature=800, pressure=0, n_ionic_steps=10000)

We can now see how pyiron sets-up the corresponding LAMMPS input

In [ ]:
job_lammps.input.control

Once the `run()` commmand is called, pyiron creates necessary input files, calls the simulation code, and finally parses and stores the output.

In [ ]:
job_lammps.run()

When printing the project, the saved job will also appear under `nodes` now.

In [ ]:
pr

You can get a quick overview with the `job_table` method.

In [ ]:
pr.job_table()

Once it is finished we can access the parsed output.

In [ ]:
job_lammps['output']

In [ ]:
plt.plot(job_lammps['output/generic/energy_pot']);

In [ ]:
job_lammps.animate_structure()

### <font style="font-family:roboto;color:#455e6c"> Software used in this notebook </font>  

- [pyiron_atomistics](https://github.com/pyiron/pyiron_atomistics)
- [LAMMPS](https://www.lammps.org/)
- [ASE](https://wiki.fysik.dtu.dk/ase/index.html)